In [2]:
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [3]:
client = bigquery.Client("ict3097")
dataset_ref = client.dataset("crypto_ethereum", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

In [4]:
sc_df = pd.read_csv('/home/malik_cyrus/ict3097_fyp/get_transactions/smart_contracts_final.csv')

In [5]:
sc_df = sc_df[['id', 'symbol', 'name', 'last_updated', 'contract_address', 'last_updated']]
sc_df

,id,symbol,name,last_updated,contract_address,last_updated
0,tether,usdt,Tether,2023-02-21T17:35:09.125Z,0xdac17f958d2ee523a2206206994597c13d831ec7,2023-02-21T17:35:09.125Z
1,binancecoin,bnb,BNB,2023-02-21T17:36:02.939Z,0xb8c77482e45f1f44de1745f52c74426c631bdd52,2023-02-21T17:36:02.939Z
2,usd-coin,usdc,USD Coin,2023-02-21T17:36:12.279Z,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2023-02-21T17:36:12.279Z
3,okb,okb,OKB,2023-02-21T17:36:19.305Z,0x75231f58b43240c9718dd58b4967c5114342a86c,2023-02-21T17:36:19.305Z
4,binance-usd,busd,Binance USD,2023-02-21T17:36:16.331Z,0x4fabb145d64652a948d72533023f6e7a623c7c53,2023-02-21T17:36:16.331Z
...,...,...,...,...,...,...
3972,zero-tech,zero,Zero Tech,2023-02-21T17:47:33.358Z,0x0ec78ed49c2d27b315d462d43b5bab94d2c79bf8,2023-02-21T17:47:33.358Z
3973,zeus10000,zeus10000,ZEUS10000,2023-02-21T17:47:33.435Z,0xcbc1aa6961b28cccb11e1cddd84ad8b42ec920b1,2023-02-21T17:47:33.435Z
3974,1million-nfts,1mil,1MillionNFTs,2023-02-21T17:45:54.423Z,0xa4ef4b0b23c1fc81d3f9ecf93510e64f58a4a016,2023-02-21T17:45:54.423Z
3975,zin,zin,Zin,2023-02-21T17:28:19.838Z,0x033e223870f766644f7f7a4b7dc2e91573707d06,2023-02-21T17:28:19.838Z


In [6]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)


amended_tokens
balances
blocks
contracts
logs
sessions
token_transfers
tokens
traces
transactions


In [7]:
sc_df['contract_address'].head()[0]

'0xdac17f958d2ee523a2206206994597c13d831ec7'

In [ ]:
query_sent_tnx = f"""
        SELECT COUNT(to_address), COUNT(DISTINCT to_address), MIN(value), MAX(value), AVG(value), SUM(value) FROM `bigquery-public-data.crypto_ethereum.transactions` 
        WHERE DATE(block_timestamp) <=  "2020-08-01" 
        AND from_address = "{sc_addr}" 
        LIMIT 100
    """ # sent_tnx, unique_sent_to_addresses, min_value_received, max_value_received, avg_value_received, total_ether_sent -- not working for normal ethereum accounts

query_received_tnx = f"""
        SELECT COUNT(from_address), COUNT(DISTINCT from_address), MIN(value), MAX(value), AVG(value), SUM(value) FROM `bigquery-public-data.crypto_ethereum.transactions` 
        WHERE DATE(block_timestamp) <=  "2020-08-01" 
        AND to_address = "{sc_addr}"
        LIMIT 100
    """  # received_tnx, unique_received_from_addresses, min_value_sent, max_value_sent, avg_value_sent, total_ether_received

query_avg_min_between_sent = f"""
        SELECT TIMESTAMPDIFF(MIN(SUB_QUERY), MAX(SUB_QUERY), MINUTE) / (COUNT(DISTINCT(SUB_QUERY)) -1) 
        FROM (
            SELECT block_timestamp FROM `bigquery-public-data.crypto_ethereum.transactions` 
            WHERE from_address = "{sc_addr}" 
            AND block_timestamp <= "2020-08-01"
        ) AS SUB_QUERY
    """ # for some reason, returns 0 for every smart contract

query_avg_min_between_received = f""" 
        SELECT TIMESTAMP_DIFF(MAX(SUB_QUERY.block_timestamp), MIN(SUB_QUERY.block_timestamp), MINUTE) / (COUNT(DISTINCT(SUB_QUERY.block_timestamp)) -1) 
        FROM (
            SELECT block_timestamp FROM `bigquery-public-data.crypto_ethereum.transactions`
            WHERE to_address = "{sc_addr}" 
            AND DATE(block_timestamp) <=  "2021-08-01"
            ) AS SUB_QUERY
    """ 

query_no_of_created_contracts = f"""
        SELECT COUNT(to_address) FROM `bigquery-public-data.crypto_ethereum.transactions` 
        WHERE DATE(block_timestamp) <=  "2020-08-01" 
        AND from_address = "{sc_addr}"
        AND to_address = NULL
        LIMIT 100
    """

query_total_ether_balance = f"""
        SELECT eth_balance FROM `bigquery-public-data.crypto_ethereum.balances`
        WHERE address = "{sc_addr}"
        LIMIT 1    
    """

# erc 20 transactions below - forget this for now 
query_total_erc20_tnx = f""" 
        SELECT COUNT(token_address) FRON `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address = "{sc_addr}"
    """

query_total_ether_received = f"""
        SELECT COUNT(token_address) FRON `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE 
    """

In [8]:
sent_tnx = []
received_tnx = [] 
unique_sent_to_addr = []
unique_received_from_addr = []
min_val_sent = []
max_val_sent = []
avg_val_sent = []
min_val_received = []
max_val_received = []
avg_val_received = []
total_ether_balance = []
no_of_created_contracts = []
total_ether_sent = []
total_ether_received = []
total_no_of_tnx = []
avg_min_between_received = []

In [ ]:
i = 0 
for sc_addr in sc_df['contract_address']:
    query_received_tnx = f"""
        SELECT COUNT(from_address), COUNT(DISTINCT from_address), MIN(value), MAX(value), AVG(value), SUM(value) FROM `bigquery-public-data.crypto_ethereum.transactions` 
        WHERE DATE(block_timestamp) <=  "2020-08-01" 
        AND to_address = "{sc_addr}"
        LIMIT 100
    """ 
    query_job = client.query(query_received_tnx)
    rows = query_job.result()  # Waits for query to finish
    df = rows.to_dataframe()
    
    received_tnx.append(df.iloc[0][0])
    unique_received_from_addr.append(df.iloc[0][1])
    try:
        min_val_received.append(float(df.iloc[0][2]))
    except TypeError:
        min_val_received.append(np.nan)
    try:
        max_val_received.append(float(df.iloc[0][3]))
    except TypeError:
        max_val_received.append(np.nan)
    try:
        avg_val_received.append(float(df.iloc[0][4]))
    except TypeError:
        avg_val_received.append(np.nan)
    try:
        total_ether_received.append(float(df.iloc[0][5]))
    except TypeError:
        total_ether_received.append(np.nan)

In [23]:
sc_df['Received_tnx'] = received_tnx

In [10]:
sc_df['Received_tnx'] = received_tnx
sc_df['Unique_Received_From_Addresses'] = unique_received_from_addr
sc_df['Min_Value_Received'] = min_val_received
sc_df['Max_Value_Received'] = max_val_received
sc_df['Avg_Value_Received'] = avg_val_received
sc_df['Total_Ether_Received'] = total_ether_received

In [11]:
sc_df
sc_df.to_csv('sc_df_with_received.csv')

In [12]:
sc_df

,id,symbol,name,last_updated,contract_address,last_updated,Received_tnx,Unique_Received_From_Addresses,Min_Value_Received,Max_Value_Received,Avg_Value_Received,Total_Ether_Received
0,tether,usdt,Tether,2023-02-21T17:35:09.125Z,0xdac17f958d2ee523a2206206994597c13d831ec7,2023-02-21T17:35:09.125Z,45732208,6424643,0.0,9.300000e+19,4.170857e+12,1.907425e+20
1,binancecoin,bnb,BNB,2023-02-21T17:36:02.939Z,0xb8c77482e45f1f44de1745f52c74426c631bdd52,2023-02-21T17:36:02.939Z,836337,178529,0.0,3.900000e+18,7.005659e+12,5.859092e+18
2,usd-coin,usdc,USD Coin,2023-02-21T17:36:12.279Z,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2023-02-21T17:36:12.279Z,2178354,765420,0.0,3.340000e+18,4.183214e+12,9.112520e+18
3,okb,okb,OKB,2023-02-21T17:36:19.305Z,0x75231f58b43240c9718dd58b4967c5114342a86c,2023-02-21T17:36:19.305Z,79678,12402,0.0,9.700000e+17,2.837672e+13,2.261000e+18
4,binance-usd,busd,Binance USD,2023-02-21T17:36:16.331Z,0x4fabb145d64652a948d72533023f6e7a623c7c53,2023-02-21T17:36:16.331Z,60819,7678,0.0,7.900000e+15,1.364705e+11,8.300000e+15
...,...,...,...,...,...,...,...,...,...,...,...,...
3972,zero-tech,zero,Zero Tech,2023-02-21T17:47:33.358Z,0x0ec78ed49c2d27b315d462d43b5bab94d2c79bf8,2023-02-21T17:47:33.358Z,0,0,NaN,NaN,NaN,NaN
3973,zeus10000,zeus10000,ZEUS10000,2023-02-21T17:47:33.435Z,0xcbc1aa6961b28cccb11e1cddd84ad8b42ec920b1,2023-02-21T17:47:33.435Z,0,0,NaN,NaN,NaN,NaN
3974,1million-nfts,1mil,1MillionNFTs,2023-02-21T17:45:54.423Z,0xa4ef4b0b23c1fc81d3f9ecf93510e64f58a4a016,2023-02-21T17:45:54.423Z,0,0,NaN,NaN,NaN,NaN
3975,zin,zin,Zin,2023-02-21T17:28:19.838Z,0x033e223870f766644f7f7a4b7dc2e91573707d06,2023-02-21T17:28:19.838Z,0,0,NaN,NaN,NaN,NaN


In [ ]:
for sc_addr in sc_df['contract_address']:
    query_total_ether_balance = f"""
        SELECT eth_balance FROM `bigquery-public-data.crypto_ethereum.balances`
        WHERE address = "{sc_addr}"
        LIMIT 1    
    """
    query_job = client.query(query_total_ether_balance)
    rows = query_job.result()
    df = rows.to_dataframe()

    total_ether_balance.append(df.iloc[0][0])
    print("Done: ", i)
    i+=1

print("Query total ether balance done")
i = 0

for sc_addr in sc_df['contract_address']:
    query_no_of_created_contracts = f"""
        SELECT COUNT(to_address) FROM `bigquery-public-data.crypto_ethereum.transactions` 
        WHERE DATE(block_timestamp) <=  "2020-08-01" 
        AND from_address = "{sc_addr}"
        AND to_address = NULL
        LIMIT 100
    """
    rows = query_job.result()  # Waits for query to finish
    df = rows.to_dataframe()

    no_of_created_contracts.append(df.iloc[0][0])
    print("Done: ", i)
    i+=1

print("Query no. of created contracts done") # returned 0 for every smart contract for some reason

i = 0
for sc_addr in sc_df['contract_address']:
    query_avg_min_between_received = f""" 
        SELECT TIMESTAMP_DIFF(MAX(SUB_QUERY.block_timestamp), MIN(SUB_QUERY.block_timestamp), MINUTE) / NULLIF((COUNT(DISTINCT(SUB_QUERY.block_timestamp)) -1), 0) 
        FROM (
            SELECT block_timestamp FROM `bigquery-public-data.crypto_ethereum.transactions`
            WHERE to_address = "{sc_addr}" 
            AND DATE(block_timestamp) <=  "2021-08-01"
            ) AS SUB_QUERY
    """ 
    query_job = client.query(query_avg_min_between_received)
    rows = query_job.result()  # Waits for query to finish
    df = rows.to_dataframe()

    avg_min_between_received.append(df.iloc[0][0])

In [34]:
for i in range(len(avg_min_between_received), len(sc_df)):
    avg_min_between_received.append(np.nan)

sc_df['Avg_Min_Between_Received'] = avg_min_between_received
sc_df

,id,symbol,name,last_updated,contract_address,last_updated,Received_tnx,Unique_Received_From_Addresses,Min_Value_Received,Max_Value_Received,Avg_Value_Received,Total_Ether_Received,Total_Ether_Balance,Avg_Min_Between_Received
0,tether,usdt,Tether,2023-02-21T17:35:09.125Z,0xdac17f958d2ee523a2206206994597c13d831ec7,2023-02-21T17:35:09.125Z,45732208,6424643,0.0,9.300000e+19,4.170857e+12,1.907425e+20,1.000000000,0.401998
1,binancecoin,bnb,BNB,2023-02-21T17:36:02.939Z,0xb8c77482e45f1f44de1745f52c74426c631bdd52,2023-02-21T17:36:02.939Z,836337,178529,0.0,3.900000e+18,7.005659e+12,5.859092e+18,13081900605247702139.000000000,5.875894
2,usd-coin,usdc,USD Coin,2023-02-21T17:36:12.279Z,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2023-02-21T17:36:12.279Z,2178354,765420,0.0,3.340000e+18,4.183214e+12,9.112520e+18,0E-9,0.481525
3,okb,okb,OKB,2023-02-21T17:36:19.305Z,0x75231f58b43240c9718dd58b4967c5114342a86c,2023-02-21T17:36:19.305Z,79678,12402,0.0,9.700000e+17,2.837672e+13,2.261000e+18,0E-9,21.493038
4,binance-usd,busd,Binance USD,2023-02-21T17:36:16.331Z,0x4fabb145d64652a948d72533023f6e7a623c7c53,2023-02-21T17:36:16.331Z,60819,7678,0.0,7.900000e+15,1.364705e+11,8.300000e+15,0E-9,2.585837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3972,zero-tech,zero,Zero Tech,2023-02-21T17:47:33.358Z,0x0ec78ed49c2d27b315d462d43b5bab94d2c79bf8,2023-02-21T17:47:33.358Z,0,0,NaN,NaN,NaN,NaN,0E-9,NaN
3973,zeus10000,zeus10000,ZEUS10000,2023-02-21T17:47:33.435Z,0xcbc1aa6961b28cccb11e1cddd84ad8b42ec920b1,2023-02-21T17:47:33.435Z,0,0,NaN,NaN,NaN,NaN,803104445672364699.000000000,NaN
3974,1million-nfts,1mil,1MillionNFTs,2023-02-21T17:45:54.423Z,0xa4ef4b0b23c1fc81d3f9ecf93510e64f58a4a016,2023-02-21T17:45:54.423Z,0,0,NaN,NaN,NaN,NaN,0E-9,NaN
3975,zin,zin,Zin,2023-02-21T17:28:19.838Z,0x033e223870f766644f7f7a4b7dc2e91573707d06,2023-02-21T17:28:19.838Z,0,0,NaN,NaN,NaN,NaN,0E-9,NaN


In [37]:
sc_df.to_csv('updated_csv.csv')

In [35]:
percent_missing = sc_df.isnull().sum() * 100 / len(sc_df)

In [36]:
percent_missing

id                                 0.000000
symbol                             0.000000
name                               0.000000
last_updated                       0.000000
contract_address                   0.000000
last_updated                       0.000000
Received_tnx                       0.000000
Unique_Received_From_Addresses     0.000000
Min_Value_Received                72.164948
Max_Value_Received                72.164948
Avg_Value_Received                72.164948
Total_Ether_Received              72.164948
Total_Ether_Balance                0.000000
Avg_Min_Between_Received          65.501634
dtype: float64

In [30]:
len(no_of_created_contracts)

3977

In [18]:
sc_df['Total_Ether_Balance'] = total_ether_balance

In [26]:
sc_df

,id,symbol,name,last_updated,contract_address,last_updated,Received_tnx,Unique_Received_From_Addresses,Min_Value_Received,Max_Value_Received,Avg_Value_Received,Total_Ether_Received,Total_Ether_Balance
0,tether,usdt,Tether,2023-02-21T17:35:09.125Z,0xdac17f958d2ee523a2206206994597c13d831ec7,2023-02-21T17:35:09.125Z,45732208,6424643,0.0,9.300000e+19,4.170857e+12,1.907425e+20,1.000000000
1,binancecoin,bnb,BNB,2023-02-21T17:36:02.939Z,0xb8c77482e45f1f44de1745f52c74426c631bdd52,2023-02-21T17:36:02.939Z,836337,178529,0.0,3.900000e+18,7.005659e+12,5.859092e+18,13081900605247702139.000000000
2,usd-coin,usdc,USD Coin,2023-02-21T17:36:12.279Z,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2023-02-21T17:36:12.279Z,2178354,765420,0.0,3.340000e+18,4.183214e+12,9.112520e+18,0E-9
3,okb,okb,OKB,2023-02-21T17:36:19.305Z,0x75231f58b43240c9718dd58b4967c5114342a86c,2023-02-21T17:36:19.305Z,79678,12402,0.0,9.700000e+17,2.837672e+13,2.261000e+18,0E-9
4,binance-usd,busd,Binance USD,2023-02-21T17:36:16.331Z,0x4fabb145d64652a948d72533023f6e7a623c7c53,2023-02-21T17:36:16.331Z,60819,7678,0.0,7.900000e+15,1.364705e+11,8.300000e+15,0E-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3972,zero-tech,zero,Zero Tech,2023-02-21T17:47:33.358Z,0x0ec78ed49c2d27b315d462d43b5bab94d2c79bf8,2023-02-21T17:47:33.358Z,0,0,NaN,NaN,NaN,NaN,0E-9
3973,zeus10000,zeus10000,ZEUS10000,2023-02-21T17:47:33.435Z,0xcbc1aa6961b28cccb11e1cddd84ad8b42ec920b1,2023-02-21T17:47:33.435Z,0,0,NaN,NaN,NaN,NaN,803104445672364699.000000000
3974,1million-nfts,1mil,1MillionNFTs,2023-02-21T17:45:54.423Z,0xa4ef4b0b23c1fc81d3f9ecf93510e64f58a4a016,2023-02-21T17:45:54.423Z,0,0,NaN,NaN,NaN,NaN,0E-9
3975,zin,zin,Zin,2023-02-21T17:28:19.838Z,0x033e223870f766644f7f7a4b7dc2e91573707d06,2023-02-21T17:28:19.838Z,0,0,NaN,NaN,NaN,NaN,0E-9


In [ ]:
i = 0
for sc_addr in sc_df['contract_address']:
    query_sent_tnx = f"""
        SELECT COUNT(to_address), COUNT(DISTINCT to_address), MIN(value), MAX(value), AVG(value), SUM(value) FROM `bigquery-public-data.crypto_ethereum.transactions` 
        WHERE DATE(block_timestamp) <=  "2020-08-01" 
        AND from_address = "{sc_addr}" 
        LIMIT 100
    """ 
    query_job = client.query(query_sent_tnx)
    rows = query_job.result()  # Waits for query to finish
    df = rows.to_dataframe()
    
    sent_tnx.append(df.iloc[0][0])
    unique_sent_to_addr.append(df.iloc[0][1])
    min_val_sent.append(float(df.iloc[0][2]))
    max_val_sent.append(float(df.iloc[0][3]))
    avg_val_sent.append(float(df.iloc[0][4]))
    total_ether_sent.append(float(df.iloc[0][5]))
    print("Done: ", i)
    i+=1


In [37]:
unique_received_from_adresses = []
unique_sent_to_addresses = []

for sc_addr in sc_df['contract_address'].head():
    # query = f"""
    #     SELECT COUNT(to_address) FROM `bigquery-public-data.crypto_ethereum.transactions` 
    #     WHERE DATE(block_timestamp) <=  "2020-08-01" 
    #     AND to_address = "{sc_addr}"
    #     LIMIT 100
    # """
    query_job = client.query(query_avg_received)  # API request
    rows = query_job.result()  # Waits for query to finish
    df = rows.to_dataframe()
    try:
        unique_received_from_adresses.append(df.iloc[0][0])
    except ValueError:
        unique_received_from_adresses.append(np.nan)



In [39]:
query = """
SELECT nonce FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE DATE(block_timestamp) = "2023-03-07" AND from_address ="0xb8c77482e45f1f44de1745f52c74426c631bdd52" LIMIT 100
"""
query_job = client.query(query)  # API request
rows = query_job.result()  # Waits for query to finish
df = rows.to_dataframe()